In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

import helper
import dataloader

In [3]:
query = """
        SELECT
            MarketName, 
            Booking, 
            DateOfBirth, 
            DateBooked, 
            WeekBooked, 
            isnull(ws.SortOrder,13) as SortOrder,
            ProgramDuration,
            DateCaxed, 
            FiscalYear, 
            ProgramStartDate, 
            SalesRepID, 
            u.Name as SalesRepName, 
            OriginalRanking, 
            MethodOfCreation,
            hasBooked =1,
            hasCaxed = case when DateCaxed is null then 0 else 1 end
        from sales s
        join Users u on u.User_id = s.SalesRepID
        left join WeekSort ws on ws.WeekNum = right(s.WeekBooked,2)
        """

In [4]:
df_original = helper.get_dataframe_from_sqlserver_query(query=query)

/Users/ganesh/Documents/ImportantWork/CleanRepo/BookingSales/helper.py:82: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [5]:
df_original.head()

,MarketName,Booking,DateOfBirth,DateBooked,WeekBooked,SortOrder,ProgramDuration,DateCaxed,FiscalYear,ProgramStartDate,SalesRepID,SalesRepName,OriginalRanking,MethodOfCreation,hasBooked,hasCaxed
0,China,CN_198974,2004-05-06,2020-06-24 21:23:37,2026,39,9.0,NaT,2020,2020-06-29,892,Malvina Belgrano,6,Website,1,0
1,Japan,JP_180044,2004-12-15,2020-07-29 20:26:36,2031,44,4.0,NaT,2020,2020-08-03,892,Malvina Belgrano,1,EnteredByUser,1,0
2,South Korea,KR_129434,2004-01-06,2020-06-24 18:52:08,2026,39,9.0,NaT,2020,2020-06-29,892,Malvina Belgrano,1,EnteredByUser,1,0
3,Vietnam,VN_133204,2004-06-06,2020-07-29 18:04:08,2031,44,4.0,NaT,2020,2020-08-03,892,Malvina Belgrano,4,EnteredByUser,1,0
4,Vietnam,VN_133206,2004-01-31,2020-07-29 18:18:49,2031,44,4.0,NaT,2020,2020-08-03,892,Malvina Belgrano,4,EnteredByUser,1,0


In [6]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34705 entries, 0 to 34704
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   MarketName        34705 non-null  object        
 1   Booking           34705 non-null  object        
 2   DateOfBirth       34703 non-null  datetime64[ns]
 3   DateBooked        34705 non-null  datetime64[ns]
 4   WeekBooked        34705 non-null  int64         
 5   SortOrder         34705 non-null  int64         
 6   ProgramDuration   34704 non-null  float64       
 7   DateCaxed         10320 non-null  datetime64[ns]
 8   FiscalYear        34705 non-null  int64         
 9   ProgramStartDate  34601 non-null  datetime64[ns]
 10  SalesRepID        34705 non-null  int64         
 11  SalesRepName      34705 non-null  object        
 12  OriginalRanking   34705 non-null  int64         
 13  MethodOfCreation  34701 non-null  object        
 14  hasBooked         3470

### Pre-Processing

In [7]:

df_bookings = (df_original
               .assign(
                       ProgramDuration = lambda _df: _df['ProgramDuration'].fillna(_df['ProgramDuration'].median()).astype('int16'),
                       DaysToCAX = lambda _df: (_df['DateCaxed'] - _df['DateBooked']).dt.days.astype('Int64'),
                       WeekNum = lambda _df: _df['WeekBooked'].astype('str').str[-2:].astype('int8'),
                       hasCaxed = lambda _df: _df['hasCaxed'].astype('bool'),
                       hasBooked = lambda _df: _df['hasBooked'].astype('bool'),
                )
               .rename(columns={'Booking': 'BookingId'})
)


In [8]:
df_bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34705 entries, 0 to 34704
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   MarketName        34705 non-null  object        
 1   BookingId         34705 non-null  object        
 2   DateOfBirth       34703 non-null  datetime64[ns]
 3   DateBooked        34705 non-null  datetime64[ns]
 4   WeekBooked        34705 non-null  int64         
 5   SortOrder         34705 non-null  int64         
 6   ProgramDuration   34705 non-null  int16         
 7   DateCaxed         10320 non-null  datetime64[ns]
 8   FiscalYear        34705 non-null  int64         
 9   ProgramStartDate  34601 non-null  datetime64[ns]
 10  SalesRepID        34705 non-null  int64         
 11  SalesRepName      34705 non-null  object        
 12  OriginalRanking   34705 non-null  int64         
 13  MethodOfCreation  34701 non-null  object        
 14  hasBooked         3470

In [1]:
data = df_bookings.groupby(['MarketName','SalesRepName'])[['BookingId', 'ProgramDuration']].agg({'BookingId':'count', 'ProgramDuration':'sum'}).reset_index()
fig = px.scatter(data, x="BookingId", y="ProgramDuration", color="MarketName", opacity= 0.8, size='ProgramDuration')
fig.show()

NameError: name 'df_bookings' is not defined

In [9]:
df_bookings.columns

Index(['MarketName', 'BookingId', 'DateOfBirth', 'DateBooked', 'WeekBooked',
       'SortOrder', 'ProgramDuration', 'DateCaxed', 'FiscalYear',
       'ProgramStartDate', 'SalesRepID', 'SalesRepName', 'OriginalRanking',
       'MethodOfCreation', 'hasBooked', 'hasCaxed', 'DaysToCAX', 'WeekNum'],
      dtype='object')

In [11]:
df_bookings[df_bookings['DateCaxed'].isna()]

,MarketName,BookingId,DateOfBirth,DateBooked,WeekBooked,SortOrder,ProgramDuration,DateCaxed,FiscalYear,ProgramStartDate,SalesRepID,SalesRepName,OriginalRanking,MethodOfCreation,hasBooked,hasCaxed,DaysToCAX,WeekNum
0,China,CN_198974,2004-05-06,2020-06-24 21:23:37,2026,39,9,NaT,2020,2020-06-29,892,Malvina Belgrano,6,Website,True,False,<NA>,26
1,Japan,JP_180044,2004-12-15,2020-07-29 20:26:36,2031,44,4,NaT,2020,2020-08-03,892,Malvina Belgrano,1,EnteredByUser,True,False,<NA>,31
2,South Korea,KR_129434,2004-01-06,2020-06-24 18:52:08,2026,39,9,NaT,2020,2020-06-29,892,Malvina Belgrano,1,EnteredByUser,True,False,<NA>,26
3,Vietnam,VN_133204,2004-06-06,2020-07-29 18:04:08,2031,44,4,NaT,2020,2020-08-03,892,Malvina Belgrano,4,EnteredByUser,True,False,<NA>,31
4,Vietnam,VN_133206,2004-01-31,2020-07-29 18:18:49,2031,44,4,NaT,2020,2020-08-03,892,Malvina Belgrano,4,EnteredByUser,True,False,<NA>,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34700,Hong Kong,HK_137037,2004-09-12,2021-09-01 17:08:59,2135,48,35,NaT,2021,2022-09-19,69130,Christy Tam,1,EnteredByUser,True,False,<NA>,35
34701,Hong Kong,HK_137039,2004-04-14,2021-09-04 17:36:27,2136,49,39,NaT,2021,2021-09-20,69130,Christy Tam,8,Website,True,False,<NA>,36
34702,Hong Kong,HK_137016,2003-12-14,2021-08-03 17:32:04,2131,44,50,NaT,2021,2021-09-20,69130,Christy Tam,6,Website,True,False,<NA>,31
34703,Hong Kong,HK_137046,2004-08-22,2021-09-18 18:10:07,2138,51,77,NaT,2021,2022-09-19,69300,Shirley Ling,5,Website,True,False,<NA>,38


In [23]:
df_bookings.columns

Index(['MarketName', 'BookingId', 'DateOfBirth', 'DateBooked', 'WeekBooked',
       'SortOrder', 'ProgramDuration', 'DateCaxed', 'FiscalYear',
       'ProgramStartDate', 'SalesRepID', 'SalesRepName', 'OriginalRanking',
       'MethodOfCreation', 'hasBooked', 'hasCaxed', 'DaysToCAX', 'WeekNum'],
      dtype='object')

In [79]:
df_booked = (df_bookings
    .rename(columns={'DateBooked': 'MainDate'})
    .assign(
        MainDate = lambda _df: _df['MainDate'].dt.normalize(),
        RankBin = lambda _df: pd.cut(df_bookings['OriginalRanking'], bins = [0,4,5,10], labels=['1-4', '5', '6-10'],),
    )
    .drop(['OriginalRanking'], axis = 1)
    
)

In [80]:
df_booked= pd.get_dummies(df_booked, columns=['MethodOfCreation', 'RankBin'], prefix = {'MethodOfCreation':'', 'RankBin':'Rank'}, prefix_sep = '')

In [81]:
df_booked = df_booked.groupby(['MainDate', 'MarketName', 'SalesRepID', 'SalesRepName', 'WeekBooked', 'WeekNum',
       'SortOrder','FiscalYear',
       ])[['hasBooked', 'hasCaxed', 'ProgramDuration', 'EnteredByUser', 'Excel Import', 'Website', 'Rank1-4', 'Rank5','Rank6-10']].sum().reset_index()

In [82]:
df_booked

,MainDate,MarketName,SalesRepID,SalesRepName,WeekBooked,WeekNum,SortOrder,FiscalYear,hasBooked,hasCaxed,ProgramDuration,EnteredByUser,Excel Import,Website,Rank1-4,Rank5,Rank6-10
0,2018-09-28,China,1997,Christy Liu,1840,40,1,2019,2,2,18,1,0,1,2,0,0
1,2018-09-28,China,50978,Qian Liu,1840,40,1,2019,1,1,8,1,0,0,1,0,0
2,2018-09-28,China,51158,Bonnie Yin,1840,40,1,2019,2,0,5,2,0,0,2,0,0
3,2018-09-28,China,51244,Cuicui Zhai,1840,40,1,2019,2,2,105,2,0,0,2,0,0
4,2018-09-28,China,53397,Ines Wang,1840,40,1,2019,4,3,10,3,0,1,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20107,2021-09-29,Japan,64660,Maya Ichikawa,2139,39,52,2021,2,0,12,1,0,1,2,0,0
20108,2021-09-29,Japan,67577,Ryoichi Yamamoto,2139,39,52,2021,1,0,6,0,0,1,1,0,0
20109,2021-09-29,Japan,68123,Shu Nozawa,2139,39,52,2021,1,0,22,1,0,0,1,0,0
20110,2021-09-29,South Korea,57612,Eric Park,2139,39,52,2021,1,0,34,1,0,0,1,0,0


In [83]:
df_booked[['hasBooked', 'hasCaxed']].sum()

hasBooked    34705
hasCaxed     10320
dtype: int64

In [119]:
df_caxed = (df_bookings[df_bookings['hasCaxed']==True]
    .rename(columns={'DateBooked': 'MainDate'})
    .assign(
        MainDate = lambda _df: _df['MainDate'].dt.normalize(),
        RankBin = lambda _df: pd.cut(df_bookings['OriginalRanking'], bins = [0,4,5,10], labels=['1-4', '5', '6-10'],),
    )
    .drop(['OriginalRanking'], axis = 1)
    
)

In [120]:
df_caxed= pd.get_dummies(df_caxed, columns=['MethodOfCreation', 'RankBin'], prefix = {'MethodOfCreation':'', 'RankBin':'Rank'}, prefix_sep = '')

In [121]:
df_caxed = df_caxed.groupby(['MainDate', 'MarketName', 'SalesRepID', 'SalesRepName', 'WeekBooked', 'WeekNum',
       'SortOrder','FiscalYear',
       ])[['hasBooked', 'hasCaxed', 'ProgramDuration', 'EnteredByUser', 'Excel Import', 'Website', 'Rank1-4', 'Rank5','Rank6-10']].sum().reset_index()

In [122]:
df_caxed.drop(['SalesRepName', 'WeekBooked','WeekNum','SortOrder','FiscalYear','hasBooked', 'hasCaxed'], axis=1, inplace=True)

In [123]:
df_caxed

,MainDate,MarketName,SalesRepID,ProgramDuration,EnteredByUser,Excel Import,Website,Rank1-4,Rank5,Rank6-10
0,2018-09-28,China,1997,18,1,0,1,2,0,0
1,2018-09-28,China,50978,8,1,0,0,1,0,0
2,2018-09-28,China,51244,105,2,0,0,2,0,0
3,2018-09-28,China,53397,8,2,0,1,2,0,1
4,2018-09-28,China,57068,12,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
6855,2021-09-23,Japan,64620,12,0,0,1,1,0,0
6856,2021-09-23,South Korea,51766,26,1,0,0,1,0,0
6857,2021-09-23,South Korea,57611,68,1,0,0,1,0,0
6858,2021-09-24,China,57957,62,2,0,0,2,0,0


In [151]:
df_merged = (df_booked.set_index(['MainDate',	'MarketName',	'SalesRepID'])
.join (df_caxed.set_index(['MainDate',	'MarketName',	'SalesRepID']), how='left', rsuffix='_cax')
.assign(
    # **{col : lambda _df: _df[col].fillna(0).astype('int16') 
            #    for col in ['ProgramDuration_cax', 'EnteredByUser_cax', 'Excel Import_cax', 'Website_cax']}
    ProgramDuration_cax = lambda _df: _df['ProgramDuration_cax'].fillna(0).astype('int16'),
    EnteredByUser_cax = lambda _df: _df['EnteredByUser_cax'].fillna(0).astype('int16'),      
    **{'Excel Import_cax' : lambda _df: _df['Excel Import_cax'].fillna(0).astype('int16')},    
    Website_cax = lambda _df: _df['Website_cax'].fillna(0).astype('int16'),
    **{'Rank1-4_cax' : lambda _df: _df['Rank1-4_cax'].fillna(0).astype('int16')},
    **{'Rank5_cax' : lambda _df: _df['Rank1-4_cax'].fillna(0).astype('int16')},
    **{'Rank6-10_cax' : lambda _df: _df['Rank1-4_cax'].fillna(0).astype('int16')},   
)
.rename(columns={'Excel Import_cax': 'ExcelImport_cax', 'Excel Import': 'ExcelImport'})
).reset_index()

In [152]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20112 entries, 0 to 20111
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   MainDate             20112 non-null  datetime64[ns]
 1   MarketName           20112 non-null  object        
 2   SalesRepID           20112 non-null  int64         
 3   SalesRepName         20112 non-null  object        
 4   WeekBooked           20112 non-null  int64         
 5   WeekNum              20112 non-null  int64         
 6   SortOrder            20112 non-null  int64         
 7   FiscalYear           20112 non-null  int64         
 8   hasBooked            20112 non-null  int64         
 9   hasCaxed             20112 non-null  int64         
 10  ProgramDuration      20112 non-null  int16         
 11  EnteredByUser        20112 non-null  uint8         
 12  ExcelImport          20112 non-null  uint8         
 13  Website              20112 non-

In [153]:
df_merged

,MainDate,MarketName,SalesRepID,SalesRepName,WeekBooked,WeekNum,SortOrder,FiscalYear,hasBooked,hasCaxed,...,Rank1-4,Rank5,Rank6-10,ProgramDuration_cax,EnteredByUser_cax,ExcelImport_cax,Website_cax,Rank1-4_cax,Rank5_cax,Rank6-10_cax
0,2018-09-28,China,1997,Christy Liu,1840,40,1,2019,2,2,...,2,0,0,18,1,0,1,2,2,2
1,2018-09-28,China,50978,Qian Liu,1840,40,1,2019,1,1,...,1,0,0,8,1,0,0,1,1,1
2,2018-09-28,China,51158,Bonnie Yin,1840,40,1,2019,2,0,...,2,0,0,0,0,0,0,0,0,0
3,2018-09-28,China,51244,Cuicui Zhai,1840,40,1,2019,2,2,...,2,0,0,105,2,0,0,2,2,2
4,2018-09-28,China,53397,Ines Wang,1840,40,1,2019,4,3,...,3,0,1,8,2,0,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20107,2021-09-29,Japan,64660,Maya Ichikawa,2139,39,52,2021,2,0,...,2,0,0,0,0,0,0,0,0,0
20108,2021-09-29,Japan,67577,Ryoichi Yamamoto,2139,39,52,2021,1,0,...,1,0,0,0,0,0,0,0,0,0
20109,2021-09-29,Japan,68123,Shu Nozawa,2139,39,52,2021,1,0,...,1,0,0,0,0,0,0,0,0,0
20110,2021-09-29,South Korea,57612,Eric Park,2139,39,52,2021,1,0,...,1,0,0,0,0,0,0,0,0,0
